<a href="https://colab.research.google.com/github/nofal82/neural-nets/blob/master/food_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
mkdir ~/.kaggle

In [13]:
cp kaggle.json ~/.kaggle/

In [14]:
! chmod 600 ~/.kaggle/kaggle.json

In [15]:
! kaggle datasets download -d trolukovich/food11-image-dataset

100% 1.08G/1.08G [00:53<00:00, 23.2MB/s]
100% 1.08G/1.08G [00:53<00:00, 21.6MB/s]


In [16]:
!unzip /content/food11-image-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: training/Soup/223.jpg   
  inflating: training/Soup/224.jpg   
  inflating: training/Soup/225.jpg   
  inflating: training/Soup/226.jpg   
  inflating: training/Soup/227.jpg   
  inflating: training/Soup/228.jpg   
  inflating: training/Soup/229.jpg   
  inflating: training/Soup/23.jpg    
  inflating: training/Soup/230.jpg   
  inflating: training/Soup/231.jpg   
  inflating: training/Soup/232.jpg   
  inflating: training/Soup/233.jpg   
  inflating: training/Soup/234.jpg   
  inflating: training/Soup/235.jpg   
  inflating: training/Soup/236.jpg   
  inflating: training/Soup/237.jpg   
  inflating: training/Soup/238.jpg   
  inflating: training/Soup/239.jpg   
  inflating: training/Soup/24.jpg    
  inflating: training/Soup/240.jpg   
  inflating: training/Soup/241.jpg   
  inflating: training/Soup/242.jpg   
  inflating: training/Soup/243.jpg   
  inflating: training/Soup/244.jpg   
  inflating: training/Soup/245.jpg   

In [58]:
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 16.5 MB/s eta 0:00:00


In [59]:
import pandas as pd
import numpy as np
# Import TensorFlow and other libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

<ipython-input-59-224081c327a5>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [42]:
# Define the model architecture
model = keras.Sequential([
# First convolutional layer with 32 filters, 3x3 kernel size, ReLU activation, and input shape of (224, 224, 3)
layers.Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
# Second convolutional layer with 64 filters, 3x3 kernel size, and ReLU activation
layers.Conv2D(64, 3, activation='relu'),
# First pooling layer with 2x2 pool size and stride of 2
layers.MaxPooling2D(2, strides=2),
# Third convolutional layer with 128 filters, 3x3 kernel size, and ReLU activation
layers.Conv2D(128, 3, activation='relu'),
# Second pooling layer with 2x2 pool size and stride of 2
layers.MaxPooling2D(2, strides=2),
# Flatten the output of the last pooling layer
layers.Flatten(),
# Fully connected layer with 256 units and ReLU activation
layers.Dense(256, activation='relu'),
# Dropout layer with dropout rate of 0.5
# layers.Dropout(0.5),
# Output layer with 11 units (one for each food category) and softmax activation
layers.Dense(11, activation='softmax')
])

In [60]:
# Define the model architecture as a function that takes a hyperparameters object as an argument
def build_model(hp):
  model = keras.Sequential([
  # First convolutional layer with 32 filters, 3x3 kernel size, ReLU activation, and input shape of (224, 224, 3)
  layers.Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
  # Second convolutional layer with a variable number of filters, 3x3 kernel size, and ReLU activation
  layers.Conv2D(hp.Int('filters_1', min_value=32, max_value=128, step=32), 3, activation='relu'),
  # First pooling layer with 2x2 pool size and stride of 2
  layers.MaxPooling2D(2, strides=2),
  # Third convolutional layer with a variable number of filters, 3x3 kernel size, and ReLU activation
  layers.Conv2D(hp.Int('filters_2', min_value=64, max_value=256, step=64), 3, activation='relu'),
  # Second pooling layer with 2x2 pool size and stride of 2
  layers.MaxPooling2D(2, strides=2),
  # Flatten the output of the last pooling layer
  layers.Flatten(),
  # Fully connected layer with a variable number of units and ReLU activation
  layers.Dense(hp.Int('units', min_value=128, max_value=512, step=128), activation='relu'),
  # Dropout layer with a variable dropout rate
  layers.Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)),
  # Output layer with 11 units (one for each food category) and softmax activation
  layers.Dense(11, activation='softmax')
  ])
  # Compile the model with a variable learning rate and Adam optimizer
  model.compile(
  loss='categorical_crossentropy',
  optimizer=keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
  metrics=['accuracy']
  )

  return model

In [52]:
# Load the data from the training/validation/evaluation folders
train_data = tf.keras.preprocessing.image_dataset_from_directory(
directory='/content/training',
labels='inferred',
label_mode='categorical',
image_size=(224, 224),
batch_size=32
)

Found 9866 files belonging to 11 classes.


In [53]:
# Define the data augmentation layer
augment_layer = tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.1)

# Apply the data augmentation layer to the training data
train_data = train_data.map(lambda x, y: (augment_layer(x), y))


In [54]:
val_data = tf.keras.preprocessing.image_dataset_from_directory(
directory='/content/validation',
labels='inferred',
label_mode='categorical',
image_size=(224, 224),
batch_size=32
)

Found 3430 files belonging to 11 classes.


In [55]:
eval_data = tf.keras.preprocessing.image_dataset_from_directory(
directory='/content/evaluation',
labels='inferred',
label_mode='categorical',
image_size=(224, 224),
batch_size=32
)

Found 3347 files belonging to 11 classes.


In [56]:
# Resize and normalize the images
resize_layer = tf.keras.layers.experimental.preprocessing.Resizing(224, 224)
rescale_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

train_data = train_data.map(lambda x, y: (resize_layer(x), y))
train_data = train_data.map(lambda x, y: (rescale_layer(x), y))

val_data = val_data.map(lambda x, y: (resize_layer(x), y))
val_data = val_data.map(lambda x, y: (rescale_layer(x), y))

eval_data = eval_data.map(lambda x, y: (resize_layer(x), y))
eval_data = eval_data.map(lambda x, y: (rescale_layer(x), y))

In [48]:
# Compile the model
model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)

In [49]:
# Train the model
model.fit(
train_data,
validation_data=val_data,
epochs=10,
batch_size=32
)

Epoch 1/10
309/309 [==============================] - 233s 662ms/step - loss: 2.7293 - accuracy: 0.1486 - val_loss: 2.2845 - val_accuracy: 0.1458
Epoch 2/10
309/309 [==============================] - 194s 624ms/step - loss: 2.2401 - accuracy: 0.1956 - val_loss: 2.1882 - val_accuracy: 0.2289
Epoch 3/10
309/309 [==============================] - 193s 622ms/step - loss: 2.1717 - accuracy: 0.2282 - val_loss: 2.1400 - val_accuracy: 0.2350
Epoch 4/10
309/309 [==============================] - 193s 621ms/step - loss: 2.1452 - accuracy: 0.2352 - val_loss: 2.0999 - val_accuracy: 0.2411
Epoch 5/10
309/309 [==============================] - 195s 628ms/step - loss: 2.1127 - accuracy: 0.2513 - val_loss: 2.0681 - val_accuracy: 0.2534
Epoch 6/10
309/309 [==============================] - 195s 629ms/step - loss: 2.0671 - accuracy: 0.2631 - val_loss: 2.0131 - val_accuracy: 0.2802
Epoch 7/10
309/309 [==============================] - 194s 623ms/step - loss: 2.0071 - accuracy: 0.2861 - val_loss: 2.0291 -

In [ ]:
# Evaluate the model
model.evaluate(eval_data)

In [61]:
# Define the random search tuner with the model function, the objective metric, and the maximum number of trials
tuner = RandomSearch(
build_model,
objective='val_accuracy',
max_trials=10
)

In [62]:
# Search for the best hyperparameters by training different models on the training and validation data
tuner.search(
train_data,
validation_data=val_data,
epochs=10,
batch_size=32
)

Trial 10 Complete [00h 19m 11s]
val_accuracy: 0.5198251008987427

Best val_accuracy So Far: 0.524781346321106
Total elapsed time: 02h 35m 54s


### trying pre-trained Model

In [63]:
# Load the pre-trained model (ResNet50) without the top layer
base_model = keras.applications.ResNet50(
weights='imagenet',
input_shape=(224, 224, 3),
include_top=False
)

94765736/94765736 [==============================] - 5s 0us/step


In [64]:
# Freeze the base model
base_model.trainable = False

In [70]:
# Define the model architecture with the base model and a new top layer
model = keras.Sequential([
# Use the base model as the first layer
base_model,
# Add a global average pooling layer to reduce the feature map size
layers.GlobalAveragePooling2D(),
# Add a fully connected layer with 256 units and ReLU activation
layers.Dense(256, activation='relu'),
# Add a dropout layer with dropout rate of 0.2
layers.Dropout(0.2),
# Add an output layer with 11 units (one for each food category) and softmax activation
layers.Dense(11, activation='softmax')
])


In [71]:
# Load the data from the training/validation/evaluation folders
train_data = tf.keras.preprocessing.image_dataset_from_directory(
directory='training',
labels='inferred',
label_mode='categorical',
image_size=(224, 224),
batch_size=32
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
directory='validation',
labels='inferred',
label_mode='categorical',
image_size=(224, 224),
batch_size=32
)

eval_data = tf.keras.preprocessing.image_dataset_from_directory(
directory='evaluation',
labels='inferred',
label_mode='categorical',
image_size=(224, 224),
batch_size=32
)


Found 9866 files belonging to 11 classes.
Found 3430 files belonging to 11 classes.
Found 3347 files belonging to 11 classes.


In [72]:
# Resize and normalize the images
resize_layer = tf.keras.layers.experimental.preprocessing.Resizing(224, 224)
rescale_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

train_data = train_data.map(lambda x, y: (resize_layer(x), y))
train_data = train_data.map(lambda x, y: (rescale_layer(x), y))

val_data = val_data.map(lambda x, y: (resize_layer(x), y))
val_data = val_data.map(lambda x, y: (rescale_layer(x), y))

eval_data = eval_data.map(lambda x, y: (resize_layer(x), y))
eval_data = eval_data.map(lambda x, y: (rescale_layer(x), y))

# Define the data augmentation layer
augment_layer = tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.1)

# Apply the data augmentation layer to the training data
train_data = train_data.map(lambda x, y: (augment_layer(x), y))

In [73]:
# Compile the model
model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)

In [74]:
# Train the model
model.fit(
train_data,
validation_data=val_data,
epochs=10,
batch_size=32
)

Epoch 1/10
309/309 [==============================] - 82s 249ms/step - loss: 2.2686 - accuracy: 0.1865 - val_loss: 2.1940 - val_accuracy: 0.2242
Epoch 2/10
309/309 [==============================] - 77s 247ms/step - loss: 2.2042 - accuracy: 0.2160 - val_loss: 2.1567 - val_accuracy: 0.2242
Epoch 3/10
309/309 [==============================] - 78s 251ms/step - loss: 2.1847 - accuracy: 0.2284 - val_loss: 2.1419 - val_accuracy: 0.2242
Epoch 4/10
309/309 [==============================] - 77s 247ms/step - loss: 2.1702 - accuracy: 0.2252 - val_loss: 2.1425 - val_accuracy: 0.2402
Epoch 5/10
309/309 [==============================] - 77s 247ms/step - loss: 2.1617 - accuracy: 0.2297 - val_loss: 2.1453 - val_accuracy: 0.2321
Epoch 6/10
309/309 [==============================] - 86s 275ms/step - loss: 2.1542 - accuracy: 0.2366 - val_loss: 2.1175 - val_accuracy: 0.2472
Epoch 7/10
309/309 [==============================] - 86s 276ms/step - loss: 2.1355 - accuracy: 0.2437 - val_loss: 2.1073 - val_ac

## trying new way

In [75]:

# Import tensorflow and other libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os

In [76]:
# Define the class names and labels
class_names = ["Bread", "Dairy product", "Dessert", "Egg", "Fried food", "Meat", "Noodles-Pasta", "Rice", "Seafood", "Soup", "Vegetable-Fruit"]
num_classes = len(class_names)


In [77]:
# Define the image size and batch size
img_height = 224
img_width = 224
batch_size = 32

# Create a data generator that rescales the images and applies data augmentation
data_gen = keras.preprocessing.image.ImageDataGenerator(
rescale=1./255,
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip=True
)

In [79]:
# Load the training data from the training folder
train_data = data_gen.flow_from_directory(
directory="training",
target_size=(img_height, img_width),
batch_size=batch_size,
class_mode="categorical",
shuffle=True
)
# Load the validation data from the validation folder
val_data = data_gen.flow_from_directory(
directory="validation",
target_size=(img_height, img_width),
batch_size=batch_size,
class_mode="categorical",
shuffle=False
)

# Load the evaluation data from the evaluation folder
eval_data = data_gen.flow_from_directory(
directory="evaluation",
target_size=(img_height, img_width),
batch_size=batch_size,
class_mode="categorical",
shuffle=False
)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
Found 3347 images belonging to 11 classes.


In [80]:
# Create a CNN model using keras Sequential API
model = keras.Sequential([
# Convolutional layer with 32 filters, 3x3 kernel size, ReLU activation and input shape
layers.Conv2D(32, (3, 3), activation="relu", input_shape=(img_height, img_width, 3)),
# Max pooling layer with 2x2 pool size
layers.MaxPooling2D((2, 2)),
# Convolutional layer with 64 filters, 3x3 kernel size and ReLU activation
layers.Conv2D(64, (3, 3), activation="relu"),
# Max pooling layer with 2x2 pool size
layers.MaxPooling2D((2, 2)),
# Convolutional layer with 128 filters, 3x3 kernel size and ReLU activation
layers.Conv2D(128, (3, 3), activation="relu"),
# Max pooling layer with 2x2 pool size
layers.MaxPooling2D((2, 2)),
# Flatten layer to convert the feature maps into a 1D vector
layers.Flatten(),
# Dense layer with 256 units and ReLU activation
layers.Dense(256, activation="relu"),
# Dropout layer with 0.5 rate to reduce overfitting
layers.Dropout(0.5),
# Output layer with num_classes units and softmax activation for multi-class classification
layers.Dense(num_classes, activation="softmax")
])

In [81]:
# Compile the model with categorical crossentropy loss, Adam optimizer and accuracy metric
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Print the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 26, 26, 128)     

In [ ]:
# Train the model for 10 epochs using train_data and val_data
model.fit(train_data, epochs=10, validation_data=val_data)

Epoch 1/10
309/309 [==============================] - 210s 670ms/step - loss: 2.3123 - accuracy: 0.1738 - val_loss: 2.2083 - val_accuracy: 0.2102
Epoch 2/10
309/309 [==============================] - 205s 665ms/step - loss: 2.1346 - accuracy: 0.2382 - val_loss: 1.9912 - val_accuracy: 0.3050
Epoch 3/10
309/309 [==============================] - 203s 658ms/step - loss: 2.0035 - accuracy: 0.2897 - val_loss: 1.9437 - val_accuracy: 0.3233
Epoch 4/10
309/309 [==============================] - 205s 665ms/step - loss: 1.9449 - accuracy: 0.3059 - val_loss: 1.9440 - val_accuracy: 0.3026
Epoch 5/10
309/309 [==============================] - 204s 660ms/step - loss: 1.8894 - accuracy: 0.3412 - val_loss: 1.8299 - val_accuracy: 0.3504
Epoch 6/10
309/309 [==============================] - 206s 667ms/step - loss: 1.8397 - accuracy: 0.3600 - val_loss: 1.8130 - val_accuracy: 0.3636
Epoch 7/10
  1/309 [..............................] - ETA: 4:04 - loss: 1.6974 - accuracy: 0.3125

In [ ]:
# Evaluate the model on eval_data and print the loss and accuracy
loss, acc = model.evaluate(eval_data)
print(f"Loss: {loss}, Accuracy: {acc}")